In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/widsdatathon2020/training_v2.csv
/kaggle/input/widsdatathon2020/samplesubmission.csv
/kaggle/input/widsdatathon2020/unlabeled.csv
/kaggle/input/widsdatathon2020/WiDS Datathon 2020 Dictionary.csv
/kaggle/input/widsdatathon2020/solution_template.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [23]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [4]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178
['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178


In [30]:
train.drop(train.loc[train['hospital_admit_source']=='Observation'].index, inplace=True)
train.drop(train.loc[train['gender'].isna()].index, inplace=True)

In [6]:
for col in train_categorical_cols:
    print('\n' + col)
    print(train[col].unique())


ethnicity
['Caucasian' nan 'Hispanic' 'African American' 'Asian' 'Native American'
 'Other/Unknown']

gender
['M' 'F' nan]

hospital_admit_source
['Floor' 'Emergency Department' 'Operating Room' nan 'Direct Admit'
 'Other Hospital' 'Other ICU' 'ICU to SDU' 'Recovery Room'
 'Chest Pain Center' 'Step-Down Unit (SDU)' 'Acute Care/Floor' 'PACU'
 'ICU' 'Other']

icu_admit_source
['Floor' 'Accident & Emergency' 'Operating Room / Recovery'
 'Other Hospital' 'Other ICU' nan]

icu_stay_type
['admit' 'readmit' 'transfer']

icu_type
['CTICU' 'Med-Surg ICU' 'CCU-CTICU' 'Neuro ICU' 'MICU' 'SICU'
 'Cardiac ICU' 'CSICU']

apache_3j_bodysystem
['Sepsis' 'Respiratory' 'Metabolic' 'Cardiovascular' 'Trauma'
 'Neurological' 'Gastrointestinal' 'Genitourinary' nan 'Hematological'
 'Musculoskeletal/Skin' 'Gynecological']

apache_2_bodysystem
['Cardiovascular' 'Respiratory' 'Metabolic' 'Trauma' 'Neurologic'
 'Gastrointestinal' 'Renal/Genitourinary' nan 'Undefined diagnoses'
 'Haematologic' 'Undefined Diagnos

In [31]:
train[train['gender'].isna()].shape

(0, 186)

In [18]:
#train[train['apache_2_bodysystem'] == 'Undefined diagnoses'].count()
train.loc[train['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'

encounter_id                     4112
patient_id                       4112
hospital_id                      4112
hospital_death                   4112
age                              4040
bmi                              3993
elective_surgery                 4112
ethnicity                        4044
gender                           4112
height                           4087
hospital_admit_source            3280
icu_admit_source                 4111
icu_id                           4112
icu_stay_type                    4112
icu_type                         4112
pre_icu_los_days                 4112
readmission_status               4112
weight                           4005
albumin_apache                   1187
apache_2_diagnosis               4112
apache_3j_diagnosis              4112
apache_post_operative            4112
arf_apache                       4112
bilirubin_apache                 1043
bun_apache                       3277
creatinine_apache                3416
fio2_apache 

In [ ]:
train['apache_2_bodysystem'] = train['apache_2_bodysystem'].fillna("Undefined diagnoses")

In [ ]:
train['ethnicity'] = train['ethnicity'].fillna("Other/Unknown")

In [26]:
train['hospital_admit_source'] = train['hospital_admit_source'].fillna("Other")

In [27]:
train['icu_admit_source'] = train['icu_admit_source'].fillna("Other ICU")

In [ ]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("")
test[test_categorical_cols] = test[test_categorical_cols].fillna("")

In [ ]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
outputcols = allcols[3]

In [ ]:
X = train[inputcols]
y = train[outputcols]

In [ ]:
train_pool = Pool(data=X,label = y,cat_features=train_categorical_cols,
#                   baseline= X_train[""], ## 
#                   group_id = X_train['hospital_id']
                 )

In [ ]:

predictions = []
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.04,depth = 10, loss_function='Logloss',boost_from_average=True, iterations=1000, random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])

In [ ]:
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("2503.csv",index=False)

In [ ]:
print('completed')